## My setup and workflow

This is my first post so before I dive into the content I thought it would be a good idea to give a short background on my setup and workflow. I do most of my work in linux (Ubuntu 14.04) and run Python 3.5. Most of my code development is done via text editor (Sublime), command line and IPython. If I need to share or demo code then I use Jupyter but I don't use it much when I'm actually developing because it feels a bit clunky.

## Making a request

First we need an API to make a call to. I'm going to use the EIA API because I follow energy markets and the API has pretty decent documentation. Before making out request we have to obtain an API key which is used to autheticate our request. We are also going to have to import some packages to help us make the request and parse out the information that is returned.

In [3]:
import requests
import json
import pandas as pd

The requests library gives us an easy way to send a HTTP/1.1 request the API. We'll use the json library to convert the text we receive from the API to a json object which we will then parse into a pandas dataframe.

Let's start by defining a class for our API call. If you're like me and you come from more of a functional programming background then classes may seem a bit foreign. We could use functions to get the same end result but classes offer a more logical way to keep track of state and state attributes. For example in our API call we might want to access the API response, the json, the dataframe and all the parameters that we passed to make a request. In a class we can create attributes to easily get these attributes without have to call a bunch of linked functions.

In [19]:
class GetEnergy(object):
    '''
    A class to call EIA API, capture response and parse to df
    '''
    
    # setting as class variable because same url will be used for all calls
    eia_url = 'http://api.eia.gov/series/'
    
    def __init__(self, api_key, series_id):
        '''
        :param api_key: an valid API key provided by EIA
        :param series: a valid EIA series ID
        '''
        # setting as instance variables because these can change each call
        self.api_key = api_key
        self.series_id = series_id
        self.response = self.get_response()
        
    def get_response(self):
        '''
         Calls the EIA API and returns response object
        '''
        # creating a tuple of tuples which we will pass into our api request
        api_parms = (
            ('api_key', self.api_key),
            ('series', self.series_id),
        )
        # using the request library to call API and capture response
        return requests.get(self.eia_url, params=api_parms)

I added a bit of commentary to the code but to walk through at a high level we are capturing the api_key and series id passed into the GetEnergy class, packing these into a tuple of tuples which we then unload using the request package. The request package will turn this into a url which the API will translate and use to send a response.

Let's try it out.

In [11]:
# define out input veriables here because we will probably reuse
# in practice you would want to keep the API key somewhere safe
api_key = eia_api = '5F4109570C68FDE20F42C25F5152D879'
series_id = 'EBA.SCL-ALL.D.H'
scl_load = GetEnergy(api_key, series_id)

We only defined the class to return us an API response so lets look at what we get back from the API before we do anything with it.

In [15]:
# print out the response
scl_load.response

<Response [500]>

You can lookup what different response codes mean but 200 is good - it means that our request was at least accepted by the API. Now lets take a look at the actual text that was sent to us by the API.

## Parsing the JSON

In [16]:
scl_load.response.text

''

Ok that looks like json that we are after so lets add a function to our GetEnergy class that will convert the text to a json object that we can work with.

In [17]:
def get_json(self):
        '''
         Returns a json object from EIA resposne
        '''
        return json.loads(self.response.text)

And we will need to call this in the __init__ so the full GetEnergy class now looks like

In [ ]:
class GetEnergy(object):
    '''
    A class to call EIA API, capture response and parse to df
    '''
    
    # setting as class variable because same url will be used for all calls
    eia_url = 'http://api.eia.gov/series/'
    
    def __init__(self, api_key, series_id):
        '''
        :param api_key: an valid API key provided by EIA
        :param series: a valid EIA series ID
        '''
        # setting as instance variables because these can change each call
        self.api_key = api_key
        self.series_id = series_id
        self.response = self.get_response()
        self.json = self.get_json()
        
    def get_response(self):
        '''
         Calls the EIA API and returns response object
        '''
        # creating a tuple of tuples which we will pass into our api request
        api_parms = (
            ('api_key', self.api_key),
            ('series', self.series_id),
        )
        # using the request library to call API and capture response
        return requests.get(self.eia_url, params=api_parms)
    
    def get_json(self):
        '''
         Returns a json object from EIA resposne
        '''
        return json.loads(self.response.text)